>>>>>>>>>   <p style="font-size:50px"><b> Keras Tuner

Keras Tuner Introduction ::: https://keras-team.github.io/keras-tuner/

In [3]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.preprocessing import StandardScaler

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

## Concatenate the Data Frames
X=pd.concat([X,geography,gender],axis=1)

## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Sanydardization
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [4]:
X_train.shape

(8000, 11)

In [5]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

 #####################  OR ##############################

def build_model2(hp):
    model = keras.Sequential()
    n_layers = hp.Int('num_layers', 2, 20)
    for i in range(n_layers): # No. of hidden layers
        n_neurons = hp.Int('units_'+str(i) ,min_value=32,max_value=512,step=32) # Number of neurons
        model.add(layers.Dense(units=n_neurons ,activation='relu', kernel_initializer = 'he_uniform'))

    model.add(layers.Dense(1, activation='sigmoid',kernel_initializer = 'glorot_uniform'))

    learning_rate_adam =  hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    optimizer_adam = keras.optimizers.Adam(learning_rate_adam)

    model.compile(optimizer=optimizer_adam,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model


In [6]:

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',  # we can see this in model_history.history.keys(), and when we provide the validation dataset
    max_trials=5, # How many cominations to try
    executions_per_trial=3, # How many time should the model be trainde for each combination, to reduce the variance in the results
    directory='project1',
    project_name='Churn')

`Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).`

In [12]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [13]:
tuner.search(X_train, y_train,epochs=5,validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.8631666501363119

Best val_accuracy So Far: 0.8653333385785421
Total elapsed time: 00h 03m 08s
INFO:tensorflow:Oracle triggered exit


In [33]:
tuner.results_summary()

Results summary
Results in project1\Churn
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 15
units_0: 512
units_1: 448
learning_rate: 0.0001
units_2: 480
units_3: 192
units_4: 96
units_5: 64
units_6: 288
units_7: 32
units_8: 352
units_9: 256
units_10: 96
units_11: 128
units_12: 160
units_13: 32
units_14: 32
Score: 0.8653333385785421
Trial summary
Hyperparameters:
num_layers: 6
units_0: 160
units_1: 64
learning_rate: 0.001
units_2: 64
units_3: 288
units_4: 32
units_5: 288
units_6: 64
units_7: 128
units_8: 64
units_9: 192
units_10: 256
units_11: 320
units_12: 32
units_13: 256
units_14: 480
Score: 0.8631666501363119
Trial summary
Hyperparameters:
num_layers: 13
units_0: 320
units_1: 416
learning_rate: 0.001
units_2: 256
units_3: 160
units_4: 416
units_5: 320
units_6: 160
units_7: 448
units_8: 96
units_9: 32
units_10: 32
units_11: 32
units_12: 32
Score: 0.862499992052714
Trial summary
Hyperparameters:
num_layers: 12
units_0:

In [22]:

best_hyper = tuner.get_best_hyperparameters(num_trials=2)
best_hyper

# num_trials: (int, optional). Number of HyperParameters objects to return. 
# HyperParameters will be returned in sorted order based on trial performance.

# Here as we have max_trials = 5. Therefore we have only 5 combinations of hyperparameters that are trained. So even if you put num_trails > 5, it will still return only the trained/tried 5 hyperparameters combination. 

In [30]:
best_model = tuner.hypermodel.build(best_hyper[0])
# This is to train the model on the best hyper-parameters
best_model

In [25]:
tuner.get_best_models(num_models=2)

# num_models (int, optional). Number of best models to return. 
# Models will be returned in sorted order. Defaults to 1.

# Returns:
# List of trained model instances.

`This method is only a convenience shortcut. For best performance, It is recommended to retrain your Model on the full dataset using the best hyperparameters found during search.`
i.e. the best_hyper method

In [32]:
history = best_model.fit(X_train, y_train,validation_split=0.33, batch_size = 100,epochs = 100)

Epoch 1/100
54/54 [==============================] - 5s 32ms/step - loss: 0.6597 - accuracy: 0.8008 - val_loss: 0.4892 - val_accuracy: 0.7955
Epoch 2/100
54/54 [==============================] - 1s 22ms/step - loss: 0.4797 - accuracy: 0.7842 - val_loss: 0.4369 - val_accuracy: 0.7955
Epoch 3/100
54/54 [==============================] - 1s 18ms/step - loss: 0.4368 - accuracy: 0.7902 - val_loss: 0.4197 - val_accuracy: 0.7955
Epoch 4/100
54/54 [==============================] - 1s 19ms/step - loss: 0.3882 - accuracy: 0.7984 - val_loss: 0.3915 - val_accuracy: 0.8376
Epoch 5/100
54/54 [==============================] - 1s 19ms/step - loss: 0.3651 - accuracy: 0.8591 - val_loss: 0.3802 - val_accuracy: 0.8448
Epoch 6/100
54/54 [==============================] - 1s 18ms/step - loss: 0.3473 - accuracy: 0.8655 - val_loss: 0.3737 - val_accuracy: 0.8402
Epoch 7/100
54/54 [==============================] - 1s 16ms/step - loss: 0.3319 - accuracy: 0.8643 - val_loss: 0.3704 - val_accuracy: 0.8463
Epoch 

In [35]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               6144      
_________________________________________________________________
dense_1 (Dense)              (None, 448)               229824    
_________________________________________________________________
dense_2 (Dense)              (None, 480)               215520    
_________________________________________________________________
dense_3 (Dense)              (None, 192)               92352     
_________________________________________________________________
dense_4 (Dense)              (None, 96)                18528     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                6208      
_________________________________________________________________
dense_6 (Dense)              (None, 288)               1